# Setup and data loading

In [1]:
import sys
sys.path.append("../")  # path contains python_file.py

import iris
import numpy as np
import scipy.stats

In [2]:
%load_ext autoreload
%autoreload 2

# 1. Data import and pre-processing

In [3]:
from cf_units import num2date

def get_dates(x):
    time_dimension = x.coords()[2]
    dates = time_dimension.units.num2date(time_dimension.points)
    return dates

def preprocess_isimip_evaluation_testing_data(variable):
    
    obs_all = iris.load_cube("testing_data/ISIMIP/"+variable+"_obs-hist_coarse_1979-2014.nc")
    cm = iris.load_cube("testing_data/ISIMIP/"+variable+"_sim-hist_coarse_1979-2014.nc")
    
    obs = obs_all[:, :, 0:9000]
    obs_validate = obs_all[:, :, 9001:]
    cm_hist = cm[:, :, 0:9000]
    cm_future = cm[:, :, 9001:]

    dates = {
        "time_obs_hist": get_dates(obs),
        #"obs_validate": get_dates(obs_validate),
        "time_cm_hist": get_dates(cm_hist),
        "time_cm_future": get_dates(cm_future),
    }
    
    obs = np.transpose(np.array(obs.data), (2, 0, 1))
    obs_validate = np.transpose(np.array(obs_validate.data), (2, 0, 1))
    cm_hist = np.transpose(np.array(cm_hist.data), (2, 0, 1))
    cm_future = np.transpose(np.array(cm_future.data), (2, 0, 1))
    
    return obs, obs_validate, cm_hist, cm_future, dates

In [4]:
tas_obs, tas_obs_validate, tas_cm_hist, tas_cm_future, tas_dates = preprocess_isimip_evaluation_testing_data('tas')
pr_obs, pr_obs_validate, pr_cm_hist, pr_cm_future, pr_dates = preprocess_isimip_evaluation_testing_data('pr')

# 2. Overview of implemented debiasers

In [5]:
from scipy.stats import norm, laplace, gamma

from PACKAGE_NAME.debias import QuantileMapping, QuantileDeltaMapping
from PACKAGE_NAME.debias import ScaledDistributionMapping, LinearScaling
from PACKAGE_NAME.debias import EquidistantCDFMatching, DeltaChange, ISIMIP

from PACKAGE_NAME.utils import gen_PrecipitationHurdleModel, gen_PrecipitationGammaLeftCensoredModel

from PACKAGE_NAME.variables import Temperature, Precipitation

### Debiaser 1: Delta change (DC)

Initialized from variable

In [23]:
tas_debiaser = DeltaChange.from_variable(variable = "tas")
tas_debiased_DC = tas_debiaser.apply(tas_obs, tas_cm_hist, tas_cm_future)

----- Running debiasing -----


100%|███████████████████████████████████████████| 4/4 [00:00<00:00, 2186.81it/s]


In [24]:
tas_debiaser

DeltaChange(delta_type='additive', variable='Temperature')

### Debiaser 2: Linear Scaling (LS)

Initialized from constructor

In [7]:
tas_debiaser = LinearScaling(delta_type = "additive")
tas_debiased_LS = tas_debiaser.apply(tas_obs, tas_cm_hist, tas_cm_future)

----- Running debiasing -----


100%|███████████████████████████████████████████| 4/4 [00:00<00:00, 4236.67it/s]


Two ways to initialize the same debiaser

In [8]:
LinearScaling(delta_type = "additive") == LinearScaling.from_variable("tas")

True

### Debiaser 3: Quantile Mapping (QM)

**Temperature**

Initialized from constructor

In [9]:
tas_debiaser_QM1 = QuantileMapping(distribution=norm, delta_type="additive")
tas_debiased_QM1 = tas_debiaser_QM1.apply(tas_obs, tas_cm_hist, tas_cm_future)

----- Running debiasing -----


100%|████████████████████████████████████████████| 4/4 [00:00<00:00, 874.27it/s]


Initialized from variable

In [10]:
tas_debiaser_QM2 = QuantileMapping.from_variable(variable="tas")
tas_debiased_QM2 = tas_debiaser_QM2.apply(tas_obs, tas_cm_hist, tas_cm_future)

----- Running debiasing -----


100%|████████████████████████████████████████████| 4/4 [00:00<00:00, 889.66it/s]


Compare the two

In [11]:
tas_debiaser_QM1 == tas_debiaser_QM2

True

Initialize from variable, overwrite some arguments

In [12]:
tas_debiaser_QM3 = QuantileMapping.from_variable(variable="tas", delta_type="no_delta", distribution = laplace)
tas_debiased_QM3 = tas_debiaser_QM3.apply(tas_obs, tas_cm_hist, tas_cm_future)

----- Running debiasing -----


100%|████████████████████████████████████████████| 4/4 [00:00<00:00, 731.51it/s]


Using the variable class but overwriting the standard distribution

In [13]:
tas_debiaser_QM4 = QuantileMapping.from_variable(variable=Temperature, delta_type="additive", distribution = laplace)

**Precipitation**

Using special initialisation for precipitation

In [14]:
pr_debiaser_QM1 = QuantileMapping.for_precipitation(delta_type = "multiplicative", 
                                                    precipitation_model_type = "censored", 
                                                    precipitation_censoring_value = 0.1)

Specifying all the arguments and an own model for the distribution (child-class of StatisticalModel needing only fit, cdf and ppf method) 1

In [15]:
left_censored_model = gen_PrecipitationGammaLeftCensoredModel(censoring_value = 0.1)

pr_debiaser_QM2 = QuantileMapping(delta_type = "multiplicative", distribution = left_censored_model)

pr_debiased_QM2 = pr_debiaser_QM2.apply(pr_obs, pr_cm_hist, pr_cm_future)

----- Running debiasing -----


  0%|                                                     | 0/4 [00:00<?, ?it/s]/Users/fionaspuler/Documents/GitHub/isimipy/notebooks/../PACKAGE_NAME/utils/_math_utils.py:329: RuntimeWarning: invalid value encountered in log
  return -np.sum(scipy.stats.gamma.logpdf(x, a=params[0], scale=params[1])) - nr_censored_x * np.log(
100%|█████████████████████████████████████████████| 4/4 [00:00<00:00,  9.47it/s]


Using the for_precipitation method but an own model for the distribution (child-class of StatisticalModel needing only fit, cdf and ppf method) 2

In [16]:
hurdle_model = gen_PrecipitationHurdleModel(distribution = gamma)

pr_debiaser_QM3 = QuantileMapping.for_precipitation(delta_type = "multiplicative", distribution = hurdle_model)

pr_debiased_QM3 = pr_debiaser_QM3.apply(pr_obs, pr_cm_hist, pr_cm_future)

----- Running debiasing -----


100%|█████████████████████████████████████████████| 4/4 [00:00<00:00,  4.78it/s]


Using a variable, but overwriting the distribution

In [17]:
hurdle_model = gen_PrecipitationHurdleModel(distribution = gamma)

pr_debiaser_QM4 = QuantileMapping.from_variable(variable = "pr", delta_type = "multiplicative", 
                                                 distribution = hurdle_model)

Compare QM3 and QM4

In [18]:
pr_debiaser_QM3 == pr_debiaser_QM4

True

### Debiaser 4: Equidistant CDF Matching (ECDFM)

In [19]:
tas_debiaser_ECDFM = EquidistantCDFMatching.from_variable(variable = "tas", distribution = norm)
tas_debiased_ECDFM = tas_debiaser_ECDFM.apply(tas_obs, tas_cm_hist, tas_cm_future)

----- Running debiasing -----


100%|████████████████████████████████████████████| 4/4 [00:00<00:00, 672.03it/s]


### Debiaser 5: Quantile Delta Mapping (Cannon 2015) (QDM):

In [20]:
tas_debiaser_QDM = QuantileDeltaMapping(distribution=norm, time_window_length=50)
tas_debiased_QDM = tas_debiaser_QDM.apply(tas_obs, tas_cm_hist, tas_cm_future)

----- Running debiasing -----


  0%|                                                     | 0/4 [00:00<?, ?it/s]/Users/fionaspuler/Documents/GitHub/isimipy/notebooks/../PACKAGE_NAME/debias/_quantile_delta_mapping.py:128: RuntimeWarning: invalid value encountered in true_divide
  cm_future_time_window
100%|█████████████████████████████████████████████| 4/4 [00:00<00:00, 58.75it/s]


### Debiaser 6: Scaled Distribution Mapping (Switanek 2017) (SDM)

In [21]:
tas_debiaser_SDM = ScaledDistributionMapping.from_variable(variable = "tas")
tas_debiased_SDM = tas_debiaser_SDM.apply(tas_obs, tas_cm_hist, tas_cm_future)

----- Running debiasing -----


100%|████████████████████████████████████████████| 4/4 [00:00<00:00, 197.61it/s]


### Debiaser 7: ISIMIP (Lange 2019) (ISIMIP)

From variable

In [22]:
tas_debiaser_ISIMIP1 = ISIMIP.from_variable(variable = 'tas')
tas_debiased_ISIMIP1 = tas_debiaser_ISIMIP1.apply(tas_obs, tas_cm_hist, tas_cm_future, **tas_dates)

----- Running debiasing -----


100%|█████████████████████████████████████████████| 4/4 [00:06<00:00,  1.69s/it]


In [28]:
test = ISIMIP(distribution = norm, trend_preservation_method="additive", detrending = False, iecdf_method = "linear", ecdf_method = "kernel_density")

In [29]:
test

ISIMIP(distribution=<scipy.stats._continuous_distns.norm_gen object at 0x7fe60e105280>, trend_preservation_method='additive', detrending=False, reasonable_physical_range=None, variable='unknown', lower_bound=inf, lower_threshold=inf, upper_bound=-inf, upper_threshold=-inf, trend_removal_with_significance_test=True, trend_transfer_only_for_values_within_threshold=True, adjust_frequencies_of_values_beyond_thresholds=True, event_likelihood_adjustment=False, nonparametric_qm=False, ecdf_method='kernel_density', iecdf_method='linear', mode_non_parametric_quantile_mapping='normal', running_window_mode=True, running_window_length=31, running_window_step_length=1)